In [33]:
import os

BASE_URL = f"http://localhost:8000/v1"

os.environ["BASE_URL"]    = BASE_URL
os.environ["OPENAI_API_KEY"] = "abc-123"   

print("Config set:", BASE_URL)

Config set: http://localhost:8000/v1


In [34]:

from pydantic_ai import Agent
from pydantic_ai.mcp import MCPServerSSE

server = server = MCPServerSSE('http://localhost:8090/sse') 

In [92]:
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider

provider = OpenAIProvider(
    base_url=os.environ["BASE_URL"],
    api_key=os.environ["OPENAI_API_KEY"],
)

agent_model = OpenAIModel("Qwen3-8B", provider=provider)

In [93]:
import json
from datetime import datetime, timezone, timedelta
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build

def retrive_calendar_events(user, start, end):
    events_list = []
    token_path = "Keys/"+user.split("@")[0]+".token"
    user_creds = Credentials.from_authorized_user_file(token_path)
    calendar_service = build("calendar", "v3", credentials=user_creds)
    events_result = calendar_service.events().list(calendarId='primary', timeMin=start,timeMax=end,singleEvents=True,orderBy='startTime').execute()
    events = events_result.get('items')
    
    for event in events : 
        attendee_list = []
        try:
            for attendee in event["attendees"]: 
                attendee_list.append(attendee['email'])
        except: 
            attendee_list.append("SELF")
        start_time = event["start"]["dateTime"]
        end_time = event["end"]["dateTime"]
        events_list.append(
            {"StartTime" : start_time, 
             "EndTime": end_time, 
             "NumAttendees" :len(set(attendee_list)), 
             "Attendees" : list(set(attendee_list)),
             "Summary" : event["summary"]})
    return events_list

In [ ]:
'[

In [94]:
event = retrive_calendar_events("userone.amd@gmail.com", '2025-07-17T00:00:00+05:30', '2025-07-16T00:00:00+05:30')
event

[]

In [ ]:
data = {
    "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
    "Datetime": "19-07-2025T12:34:55",
    "Location": "IISc Bangalore",
    "From": "userone.amd@gmail.com",
    "Attendees": [
        {
            "email": "usertwo.amd@gmail.com"
        },
        {
            "email": "userthree.amd@gmail.com"
        }
    ],
    "Subject": "Agentic AI Project Status Update",
    "EmailContent": "Hi team, let's meet on Thursday for 30 minutes to discuss the status of Agentic AI Project."
}

output = {}
output["Request_id"] = data["Request_id"]
output["Datetime"] = data["Datetime"]
output["Location"] = data["Location"]
output["Request_id"] = data["Request_id"]
output["From"] = data["From"]
output["Location"] = data["Location"]
output["Attendees"] = data["Attendees"]
for i in range(len(output["Attendees"])):
    email = output["Attendees"]["email"]
    occupied = 


In [42]:
SYSTEM_PROMPT = """

"""

inp = """
{
    "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
    "Datetime": "19-07-2025T12:34:55",
    "Location": "IISc Bangalore",
    "From": "userone.amd@gmail.com",
    "Attendees": [
        {
            "email": "usertwo.amd@gmail.com"
        },
        {
            "email": "userthree.amd@gmail.com"
        }
    ],
    "Subject": "Agentic AI Project Status Update",
    "EmailContent": "Hi team, let's meet on Thursday for 30 minutes to discuss the status of Agentic AI Project."
}
"""

In [96]:
from pydantic_ai import Agent

agent = Agent(
    model=agent_model,
    toolsets=[server],
    system_prompt = SYSTEM_PROMPT
)

In [97]:
import asyncio
from pydantic_ai.mcp import MCPServerStdio
async def run_async(prompt: str) -> str:
    async with agent.run_mcp_servers():
        result = await agent.run(prompt)
        return result.output


In [98]:
import time
import asyncio

start = time.perf_counter()
result = await run_async("current date and day of the week")
end = time.perf_counter()

print(f"Result: {result}")
print(f"Elapsed time: {end - start:.2f} seconds")

Result: 

The current date is **July 19, 2025**, and the day of the week is **Thursday**.
Elapsed time: 4.29 seconds


In [99]:
VLLM_USE_TRITON_FLASH_ATTN=0 \
vllm serve Qwen/Qwen3-8B \
    --served-model-name Qwen3-8B \
    --api-key abc-123 \
    --port 8000 \
    --enable-auto-tool-choice \
    --tool-call-parser hermes \
    --trust-remote-code

SyntaxError: invalid decimal literal (3612579796.py, line 2)

In [177]:
from datetime import datetime

def merge_intervals(intervals):
    if not intervals:
        return []

    # Sort by start time
    intervals.sort(key=lambda x: x[0])
    merged = [intervals[0]]

    for current_start, current_end in intervals[1:]:
        last_start, last_end = merged[-1]
        if current_start <= last_end:
            merged[-1] = (last_start, max(last_end, current_end))  # Merge overlapping
        else:
            merged.append((current_start, current_end))
    return merged

def get_available_time_slots(attendees, duration_minutes, start_date, end_date):

    user_availabilities = []

    for user in attendees:
        events = retrive_calendar_events(user, start_date, end_date)
        for event in events:
            start = datetime.fromisoformat(event["StartTime"])
            end = datetime.fromisoformat(event["EndTime"])
            user_availabilities.append((start, end))

    merged_unavailable = merge_intervals(user_availabilities)
    available = []
    duration = timedelta(minutes=duration_minutes)
    if merged_unavailable[0][0] -  datetime.fromisoformat(start_date)>=duration:
        available.append((start_date,merged_unavailable[0][0].strftime('%Y-%m-%d %H:%M:%S %z')))
    for i in range(1,len(merged_unavailable)):
        if (merged_unavailable[i][0] - merged_unavailable[i-1][1] )>=duration:
            available.append((merged_unavailable[i-1][1].strftime('%Y-%m-%d %H:%M:%S %z'),merged_unavailable[i][0].strftime('%Y-%m-%d %H:%M:%S %z')))
    if (datetime.fromisoformat(end_date)-merged_unavailable[-1][1])>=duration:
        available.append((merged_unavailable[-1][1].strftime('%Y-%m-%d %H:%M:%S %z'),end_date))
    slots = ""
    for s,e in available:
        slots+= (s+"-"+e+",\n ")
    return slots

In [181]:
f = example_usage(["userone.amd@gmail.com","usertwo.amd@gmail.com"],60,'2025-07-17T00:00:00+05:30', '2025-07-18T23:59:59+05:30')

In [186]:
slots = ""
for s,e in f:
    slots+= (s+"-"+e+",\n ")
slots

'2025-07-17 09:00:00 +0530-2025-07-17 18:00:00 +0530,\n 2025-07-18 17:00:00 +0530-2025-07-18 18:00:00 +0530,\n '

In [128]:
event = retrive_calendar_events("userone.amd@gmail.com", '2025-07-17T00:00:00+05:30', '2025-07-18T23:59:59+05:30')
event

[{'StartTime': '2025-07-16T18:00:00+05:30',
  'EndTime': '2025-07-17T09:00:00+05:30',
  'NumAttendees': 1,
  'Attendees': ['SELF'],
  'Summary': 'Off Hours'},
 {'StartTime': '2025-07-17T18:00:00+05:30',
  'EndTime': '2025-07-18T09:00:00+05:30',
  'NumAttendees': 1,
  'Attendees': ['SELF'],
  'Summary': 'Off Hours'},
 {'StartTime': '2025-07-18T18:00:00+05:30',
  'EndTime': '2025-07-19T09:00:00+05:30',
  'NumAttendees': 1,
  'Attendees': ['SELF'],
  'Summary': 'Off Hours'}]

In [121]:
event = retrive_calendar_events("usertwo.amd@gmail.com", '2025-07-17T00:00:00+05:30', '2025-07-18T23:59:59+05:30')
event

[{'StartTime': '2025-07-16T18:00:00+05:30',
  'EndTime': '2025-07-17T09:00:00+05:30',
  'NumAttendees': 1,
  'Attendees': ['SELF'],
  'Summary': 'Off Hours'},
 {'StartTime': '2025-07-17T18:00:00+05:30',
  'EndTime': '2025-07-18T09:00:00+05:30',
  'NumAttendees': 1,
  'Attendees': ['SELF'],
  'Summary': 'Off Hours'},
 {'StartTime': '2025-07-18T09:00:00+05:30',
  'EndTime': '2025-07-18T17:00:00+05:30',
  'NumAttendees': 1,
  'Attendees': ['SELF'],
  'Summary': 'Workshop Advance AI'},
 {'StartTime': '2025-07-18T18:00:00+05:30',
  'EndTime': '2025-07-19T09:00:00+05:30',
  'NumAttendees': 1,
  'Attendees': ['SELF'],
  'Summary': 'Off Hours'}]